**I am using only one summarized cell just because not sure how exactly Yandex & MPTI Engine works and how it will work will multiple cells **

Ideally without any doubt it have to big series of multiple cells to track down the progress, improve readibility, etc

In [ ]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re
import math

# standard function which is parsing lines into words

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

# instead of stroing data by arcticles, I will create RDD with all words separate
wiki_by_word = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).flatMap(parse_article) \
.map(lambda word: word.lower()).cache()

# loading all stop words into other rdd
stop_words = sc.textFile("/datasets/stop_words_en.txt", 16).map(lambda word: '%s' % word.lower()).collect()

# filtering all words by skipping stop words + indexing and reverting for future join
filtered = wiki_by_word.filter(lambda word : word not in stop_words).zipWithIndex() \
.map(lambda (key, index) : (index, key)).cache()

# creating variable to safe amount of all numbers
num_of_words_total = int(filtered.count())

# using list of filtered words, counting amount of their appeareance 
filtered_agg = filtered.map(lambda (key, index) : (index, 1)).reduceByKey(lambda a, b: a + b).cache()

# increasing index for tricky join to get all pairs
filtered_next_word = filtered.map(lambda (key,index): (key + 1, index)).cache()

# joining to get separated pairs
joined = filtered_next_word.join(filtered).cache()

# number of pairs
num_of_pairs = int(joined.count())

# concatenat the result into pair
joined_concat = joined.map(lambda (key, (val1, val2)): (val1 + '_' + val2, 1)).cache()

# filtering the pairs to safe only 500 occurencies
result = joined_concat.reduceByKey(lambda a, b: a + b).map(lambda (key, index) : (index, key)) \
.sortByKey(ascending=False).filter(lambda (key, index) : key > 499).cache()

# Start to play games: let's safe pairs and splitted words together
result_splitted = result.map(lambda (key, index) : (key, index, index.split("_"))).cache()

# a - count of pairs, b - pair itself, c[0] - 1st word, c[1] - 2nd word
words_with_numbers=result_splitted.map(lambda (a, b, c) : (a, b, c[0], c[1])).cache()

# this is really crazy transformaton when we join 2 times lost of words to get count of this word for future
# and calc all probabilities at once
big_rdd = words_with_numbers.map(lambda (a,b,c,d): (c,(a,b,d))).join(filtered_agg) \
.map(lambda (a,((b,c,d),e)) : (float(b)/float(num_of_pairs),c,float(e)/float(num_of_words_total),a,d)) \
.map(lambda (a,b,c,d,e): (e,(a,b,c,d))).join(filtered_agg) \
.map(lambda (a,((b,c,d,e), g)) : (b,c,d,e, float(g) / float(num_of_words_total), a)) \
.cache()

# calc PMI
big_rdd_with_pmi = big_rdd.map(lambda (a,b,c,d,e,f) : (math.log(a/(c*e)), a, b)).cache()

# calc NPMI
big_rdd_with_npmi = big_rdd_with_pmi.map(lambda (a,b,c): (a/((-1)*math.log(b)), c)) \
.sortByKey(ascending=False).cache()

# amount of elements
final_cnt = big_rdd_with_npmi.count()

# print output
for i in range(0, final_cnt):
    print '%s' % big_rdd_with_npmi.collect()[i][1]